In [3]:
# load agent from json
# create agent and write to disk

In [4]:
from __future__ import annotations
import os
import inspect
from dotenv import load_dotenv
import json, pathlib
from typing import List, Any
from agents import Agent, handoff, RunContextWrapper, Runner, RunHooks, AgentHooks
from pydantic import BaseModel

import logging
logging.basicConfig(level=logging.INFO)

In [5]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI API KEY IS NOT FOUND IN THE ENVIORMENT VARIABLES")

In [6]:
AGENTS_DB = pathlib.Path("./agent_data_base/agents.json")
AGENTS_DB.parent.mkdir(parents=True, exist_ok=True)

def _save(agent: Agent[Any]):
    db = json.loads(AGENTS_DB.read_text()) if AGENTS_DB.exists() else []
    db = [r for r in db if r["name"] != agent.name]  # overwrite by name
    db.append(
        {
            "name": agent.name,
            "instructions": agent.instructions,
            "tool_names": [t.name for t in agent.tools],
        }
    )
    AGENTS_DB.write_text(json.dumps(db, indent=2, ensure_ascii=False))

def load_all(tools: List) -> List[Agent]:
    if not AGENTS_DB.exists():
        return []
    data = json.loads(AGENTS_DB.read_text())
    agents = []
    for rec in data:
        rec_tools = [t for t in tools if t.name in rec["tool_names"]]
        agents.append(
            Agent(
                name=rec["name"],
                instructions=rec["instructions"],
                tools=rec_tools,
                hooks=LifecyclePrintHooks(), # 仍可有日志
            )
        )
    return agents

class SpawnInput(BaseModel):
    agent_name: str
    instructions: str

class LifecyclePrintHooks(AgentHooks[None]):
    async def on_start(self, ctx: RunContextWrapper[None], agent):
        print(f"▶️  「{agent.name}」进入执行")

    async def on_end(self, ctx: RunContextWrapper[None], agent, output):
        # output 可能是 None（还在进行中）或最终回答
        if inspect.isawaitable(output):
            output = await output
        print(f"⏹️  「{agent.name}」执行结束")
        

def make_auto_spawn_handoff(agent_owner: "SelfSpawningAgent"):
    async def on_handoff(ctx: RunContextWrapper[None], data: SpawnInput):
        logging.info(f"🚀 AutoSpawn: {agent_owner.name} → 生成新代理『{data.agent_name}』")
        new_agent = SelfSpawningAgent(          # 关键：生成同类
            name=data.agent_name,
            instructions=data.instructions,
            base_tools=agent_owner._base_tools,  # 让基础工具继承
        )
        _save(new_agent)
        agent_owner.add_specialist(new_agent)  
        logging.info(f"🤝 已交接到 {data.agent_name!r}")
        return new_agent

    return handoff(
        agent_owner,                            # 首参可以是自身
        on_handoff=on_handoff,
        input_type=SpawnInput,
        tool_name_override="auto_spawn",
        tool_description_override=(
            "Create a more specialized agent and delegate. "
            'Input JSON: {"agent_name": "...", "instructions": "..."}'
        ),
    )

class SelfSpawningAgent(Agent):
    """Any instance can further spawn children via auto_spawn."""

    def __init__(
        self,
        name: str,
        instructions: str,
        base_tools: List[Any] = None,
    ):
        sys_prompt = (
            instructions
            + "\n\nYou can call the `auto_spawn` handoff to create an even "
            "more specialised assistant if necessary."
        )
        super().__init__(name=name, instructions=sys_prompt, tools=base_tools or [])
        self._base_tools = base_tools or []
        self.handoffs.append(make_auto_spawn_handoff(self))
        self.handoffs.extend(load_all(self._base_tools))
        self.hooks=LifecyclePrintHooks()
        logging.info(f"👶 SelfSpawningAgent {self.name!r} 初始化完成")

    async def get_handoffs(self, run_context):
        base = [self.handoffs[0]]
        specialists = load_all(self._base_tools)
        names = [a.name for a in specialists]
        logging.info(f"💾 [{self.name}] 当前可用专家: {names}")
        self.handoffs = base + specialists
        return await super().get_handoffs(run_context)

    def add_specialist(self, agent: Agent):
        self.handoffs.append(agent)
        agent.handoff_description = f"Specialist for {agent.name}"
        print(f"📦 [{self.name}] 已添加专家 {agent.name}")

In [7]:
triage_agent = SelfSpawningAgent(
    name="Triage",
    instructions=(
        "You are a task router for *any* question.\n"
        "If no existing specialist fits, call `auto_spawn` with JSON:\n"
        '{"agent_name": "...", "instructions": "..."}'
    ),
    base_tools=[],        # 可以放 get_weather 等全局工具
)

INFO:root:👶 SelfSpawningAgent 'Triage' 初始化完成


In [8]:
triage_agent = SelfSpawningAgent(
    name="Triage",
    instructions=(
        "You are a task router for *any* question.\n"
        "If no existing specialist fits, call `auto_spawn` with JSON:\n"
        '{"agent_name": "...", "instructions": "..."}'
    ),
    base_tools=[],
)

INFO:root:👶 SelfSpawningAgent 'Triage' 初始化完成


In [9]:
q1_res = await Runner.run(triage_agent,
                     "Solve Ax = b over R^3 and explain rank–nullity theorem")

▶️  「Triage」进入执行


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


▶️  「Linear Algebra Solver」进入执行


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


⏹️  「Linear Algebra Solver」执行结束


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


In [10]:
q1_res.final_output

"To solve the equation \\(Ax = b\\), where \\(A\\) is a matrix and \\(b\\) is a vector, follow these steps:\n\n1. **Check the dimensions:** Ensure that \\(A\\) is a matrix that can be multiplied by a vector \\(x\\) to yield a vector in \\(\\mathbb{R}^3\\). This typically means \\(A\\) is \\(3 \\times n\\).\n\n2. **Determine the augmented matrix:** Form the augmented matrix \\([A | b]\\).\n\n3. **Row-reduce to echelon form:** Use Gaussian elimination to row-reduce the augmented matrix to column echelon form.\n\n4. **Identify pivot and free columns:**\n   - **Pivot columns** correspond to the basic variables.\n   - **Free columns** correspond to the free variables.\n\n5. **Back substitute to find solutions:**\n   - Express basic variables in terms of free variables.\n   - If the system is consistent, write the solution vector \\(x\\) in terms of these variables.\n\n6. **Interpret solutions:**\n   - If there is one unique solution, the system is determined.\n   - If there are infinite sol

In [11]:
q2_res = await Runner.run(triage_agent,
                     "What's the determinant of [[1,2],[3,4]]?")

▶️  「Triage」进入执行


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


▶️  「Linear Algebra Solver」进入执行


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


⏹️  「Linear Algebra Solver」执行结束


In [12]:
q2_res.final_output

'To find the determinant of the matrix \\(\\begin{bmatrix} 1 & 2 \\\\ 3 & 4 \\end{bmatrix}\\), follow these steps:\n\nFor a 2x2 matrix \\(\\begin{bmatrix} a & b \\\\ c & d \\end{bmatrix}\\), the determinant is calculated as:\n\n\\[\n\\text{det}(A) = ad - bc\n\\]\n\nApplying the formula:\n\n1. Identify \\(a = 1\\), \\(b = 2\\), \\(c = 3\\), \\(d = 4\\).\n\n2. Plug these values into the formula:\n\n\\[\n\\text{det}(A) = (1)(4) - (2)(3) = 4 - 6 = -2\n\\]\n\nSo, the determinant of the matrix \\(\\begin{bmatrix} 1 & 2 \\\\ 3 & 4 \\end{bmatrix}\\) is \\(-2\\).'